# Create, Retrieve, Update and Delete Operations in DynamoDB
## 1. Introduction
This notebook includes the following operations:
1.  create table
2.  list items in a specified table
3.  add items to table
4.  retrieve item by primary key
5.  show the table information for a specified table
6.  delete an item by primary key
7.  update the value of a column of an item specified by primary key
8.  query table using IndexName, KeyConditionExpression and filters
9.  query table using scan table and filters
10. delete a specified table
11. update table by adding global secondary index
12. upload text files to the specified table

## 2. Load packages and AWS credentials

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import boto3
from boto3.dynamodb.conditions import Key, Attr
import os
import time
import json
import decimal

aws_access_key=os.getenv("AWS_ACCESS_KEY")
aws_secret_access_key=os.getenv("AWS_SECRET_KEY")
region=os.getenv("AWS_REGION")
s3_bucket="s3://yuanecs/"

## 3. DynamoDBDataManager class for data table operations

In [15]:
class DecimalEncoder(json.JSONEncoder):
    def default(self,o):
        if isinstance(o,decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

class DynamoDBDataManager():
    def __init__(self, aws_access_key, aws_secret_access_key, region):
        self.session=boto3.Session(aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_access_key)
        self.client=self.session.client('dynamodb', region)
        self.resource=self.session.resource('dynamodb', region)
        
    def CreateTable(self, params):
        self.client.create_table(**params)
        table_name = params['TableName']
        print('Waiting for creating', table_name, '...')
        waiter = self.client.get_waiter('table_exists')
        waiter.wait(TableName=table_name)
        print(table_name+" successfully created!")
        
    def listTables(self):
        return self.client.list_tables()['TableNames']
    
    def describeTable(self,table_name):
        return self.client.describe_table(TableName=table_name)
    
    def deleteTable(self, table_name):
        self.client.delete_table(TableName=table_name)
        print('Waiting for deleting', table_name, '...')
        waiter = self.client.get_waiter('table_not_exists')
        waiter.wait(TableName=table_name)
        print(table_name+" successfully deleted!")
        
    def addItem(self,table_name,item):
        """
        Input:
          table_name: string, name of the table
          item: dictionary
        """
        table = self.resource.Table(table_name)
        
        return table.put_item(
            Item = item 
        )
    
    def getItem(self,table_name, key):
        """
        Input:
          table_name: string
          key: directionary
        """
        table = self.resource.Table(table_name)
        
        response = table.get_item(
            Key = key
        )
        return response['Item']
    
    def updateItem(self,table_name,key,col_name,val):
        table = self.resource.Table(table_name)
        
        response = table.update_item(
          Key = key,
          UpdateExpression='SET '+ col_name + ' =:val1',
          ExpressionAttributeValue={
              ':val1': val
          }  
        )
        return 'HTTPStatusCOde is: '+str(response['ResponseMetadata']['HTTPStatusCode'])
    
    def deleteItem(self, table_name, key):
        table = self.resource.Table(table_name)
        return table.delete_item(
          Key = key
        )['ResponseMetadata']['HTTPStatusCode']
    
    def importTxtFile(self, file_name, table_name, sep):
        table = self.resource.Table(table_name)
        
        df = pd.read_csv(file_name, sep=sep)
        df.columns = df.columns.str.replace("\.","_")
        
        with table.batch_writer() as batch:
            for idx, row in df.iterrows():
                tmp=dict()
                for i, c in row.items():
                    if not pd.isna(c):
                        tmp[i] = c
                batch.put_item(Item=tmp)
                
    def queryByKey(self, table_name, expr, index_name=None, filter_expr=None):
        table = self.resource.Table(table_name)
        result = []
        response = None
        
        while (response is None) or ('LastEvaluatedKey' in response):
            if index_name and filter_expr:
                response = table.query(
                IndexName = index_name,
                KeyConditionExpression = expr,
                FilterExpression = filter_expr                    
                )
            elif index_name:
                response = table.query(
                IndexName = index_name,
                KeyConditionExpression = expr    
                )
            elif filter_expr:
                response = table.query(
                KeyConditionExpression = expr,
                FilterExpression = filter_expr    
                )
            else:
                response = table.query(
                KeyConditionExpression=expr
                )
                
            results.extend(response['Items'])  
            
        return results
        # return json.dumps(results, cls=DecimalEncoder )
        
    def scanTable(self, table_name, filter_expr):
        table = self.resource.Table(table_name)
        
        response = table.scan(
            FilterExpression=filter_expr
        )
        
        return response['Items']
            
    def updateTableSchema(self, attr_def, table_name, GSI_updates):
        self.client.update_table(
            AttributeDefinitions=attr_def,
            TableName=table_name,
            GlobalSecondaryIndexUpdates=GSI_updates
        )   
    

## 4. Data Manipulations by DynamoDBDataManager class

In [16]:
# Initialize a DynamoDBDataManager instance
dynamodb = DynamoDBDataManager(aws_access_key, aws_secret_access_key, region)

### 1. Create table

In [ ]:
params = {'TableName': 'sample_test',
   'KeySchema': [{'AttributeName': 'GEO_study_ID', 'KeyType': 'HASH'},
    {'AttributeName': 'GEO_sample_ID', 'KeyType': 'RANGE'}],
    'AttributeDefinitions': [{'AttributeName': 'GEO_study_ID', 'AttributeType': 'S'},
    {'AttributeName': 'GEO_sample_ID', 'AttributeType': 'S'}],
    'ProvisionedThroughput': {'ReadCapacityUnits': 1, 'WriteCapacityUnits': 1} 
}

# create table using table definition paramters
dynamodb.createTable(params)

### 2. Add an item to table

In [17]:
item = {'GEO_study_ID': 'GSE102746',
        'GEO_sample_ID': 'GSM2745967',
        'Platform_ID': 'GPL16791',
        'Type': 'RNA',
        'Organism': 'Homo_sapiens',
        'Biomaterial': 'intestinal epithelial organoid',
        'Disease': 'ulcerative colitis',
        'Patient_number': 'CTD-111',
        'Age': 60,
        'Race': 'White',
        'Gender': 'M'
       }
dynamodb.addItem('sample_test',item)

### 3. Retrieve an item

In [ ]:
key = {'GEO_study_ID':'GSE102746',
       'GEO_sample_ID': 'GSM2745967'
      }
dynamodb.getItem('sample_test',key) 

### 4. Update an item by key and set teh value of a column

In [ ]:
dynamodb.updateItem('sample_test', key, 'Age', 30)

In [ ]:
dynamodb.getItem('sample_test',key)

As shown here, after updating the Age value from 60 to 30

### 5. Delete an item by key

In [ ]:
dynamodb.deleteItem('sample_test', key)

### 6. Import text file to DynamoDB table

In [ ]:
dynamodb.importTxtFile("SampleTable.txt", "sample_test", '\t')

### 7. Query table by keys (partition and sort keys)
Based on the table imported from the text file in section 6, we can query table using primary key

In [ ]:
queryexp = Key('GEO_study_ID').eq('GSE102746') & Key('GEO_sample_ID').eq('GSM2745968')
dynamodb.queryByKey("sample_test", queryexp)

### 8. Query table by keys and filter

In [ ]:
queryexp_1 = Key('GEO_study_ID').eq('GSE102746')
filter_expr_1 = Attr('Age').gt(75)

dynamodb.queryByKey("sample_test", queryexp_1, filter_expr=filter_expr_1)

### 9. Scan table using filters

In [ ]:
filter_expr = Attr('GEO_study_ID').eq('GSE102746') & Attr('Age').gt(75)
dynamodb.scanTable("sample_test", filter_expr)

### 10. Update table schema to add global secondary index
If we want to query the data using columns that are not defined by primary keys, we need to add extra sort keys. This can be done by adding either local or global secondary index, but usually, global secondary index is preferred. Defining global secondary index includes two parts: defining attributes, including the existing partion key, sort key and the column you want to use as the extra sort key; and then the Global Secondary Index, including IndexName, KeySchema (HASH and RANGE keys), Projection and ProvisionedThroughout.

In [18]:
attr_def = [{'AttributeName':'GEO_study_ID', 'AttributeType':'S'},
           {'AttributeName': 'GEO_sample_ID', 'AttributeType': 'S'},
           {'AttributeName': 'Age', 'Attribute': 'N'}]

GSI_def = [{'Create':{'IndexName': 'GSE_Age', 'KeySchema':[{'AttributeName': 'GEO_study_ID', 'KeyType': 'HASH'},
           {'AttributeName': 'Age', 'KeyType': 'RANGE'}], 'Projection': {'ProjectionType': 'ALL'},
           'ProvisionedThroughput': {'ReadCapacityUnits':1, 'WriteCapacityUnits': 1}           
}}]

In [ ]:
# Update dynamoDB table
dynamodb.updateTableSchema(attr_def, 'sample_test', GSI_def)

In [ ]:
# define the KeyConditionExpression and query table using the Global Secondary Index
key_expr = Key("GEO_study_ID").eq("GSE102746") & Key('Age').gt(10)
dynamodb.queryByKey("sample_test", key_expr, index_name='GSE_Age')

### 11. Describe table

In [ ]:
dynamodb.describeTable("sample_test")

### 12. Delete table

In [ ]:
dynamodb.deleteTable("sample_test")

In [19]:
pd.read_csv("SampleTable.txt", sep = "\t")

,Study_ID,Sample_ID,Platform_ID,Sample_title,Type,Organism,Biomaterial,Disease,Patient_number,Response,Treatment,Time_point,Age,Race,Gender
0,102757,2746967,GPL16791,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-110,NaN,NaN,NaN,60,White,M
1,102757,2746968,GPL16792,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-138,NaN,NaN,NaN,44,White,F
2,102757,2746969,GPL16793,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-139,NaN,NaN,NaN,58,White,M
3,102757,2746970,GPL16794,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-141,NaN,NaN,NaN,18,White,M
4,102757,2746971,GPL16795,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-145,NaN,NaN,NaN,58,White,F
5,102757,2746972,GPL16796,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-149,NaN,NaN,NaN,44,White,F
6,102757,2746973,GPL16797,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-150,NaN,NaN,NaN,40,White,F
7,102757,2746974,GPL16798,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-134,NaN,NaN,NaN,25,White,M
8,102757,2746975,GPL16799,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-123,NaN,NaN,NaN,24,White,M
9,102757,2746976,GPL16800,NaN,RNA,Homo sapiens,intestinal epithelial organoid,ulcerative colitis,C-543,NaN,NaN,NaN,50,White,M
